#

In [ ]:
from subprocess import check_output
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings 
warnings.filterwarnings('ignore')
from math import ceil
#Plots
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import confusion_matrix #Confusion matrix
from pandas.tools.plotting import parallel_coordinates
#Advanced optimization
from scipy import optimize as op

#Load Data
iris = pd.read_csv('../input/Iris.csv')
iris.head()

In [ ]:
#Visualizations

#Plot with respect to sepal length
sepalPlt = sb.FacetGrid(iris, hue="Species", size=6) \
   .map(plt.scatter, "SepalLengthCm", "SepalWidthCm")
plt.legend(loc='upper left')

In [ ]:
#plot with respect to petal length
petalPlt = sb.FacetGrid(iris, hue="Species", size=6) \
   .map(plt.scatter, "PetalLengthCm", "PetalWidthCm")
plt.legend(loc='upper left')

In [ ]:
#Sepal and Petal lengths
parallel_coordinates(iris.drop("Id", axis=1), "Species")

In [ ]:
#Data setup

Species = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
#Number of examples
m = iris.shape[0]
#Features
n = 4
#Number of classes
k = 3

X = np.zeros((m,n + 1))
y = np.array((m,1))
X[:,1] = iris['PetalLengthCm'].values
X[:,2] = iris['PetalWidthCm'].values
X[:,3] = iris['SepalLengthCm'].values
X[:,4] = iris['SepalWidthCm'].values

#Mean normalization
for j in range(n):
    X[:, j] = (X[:, j] - X[:,j].mean())

#Labels
y = iris['Species'].values

#Regularized Cost function and gradient

![#Regularized Cost][1]

![#Regularized Gradient][2]


  [1]: http://i.imgur.com/id7nVp6.png
  [2]: http://i.imgur.com/7x8omie.png

In [ ]:
#Logistic Regression

def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

#Regularized cost function
def regCostFunction(theta, X, y, _lambda = 0.1):
    m = len(y)
    h = sigmoid(X.dot(theta))
    tmp = np.copy(theta)
    tmp[0] = 0 
    reg = (_lambda/(2*m)) * np.sum(tmp**2)

    return (1 / m) * (-y.T.dot(np.log(h)) - (1 - y).T.dot(np.log(1 - h))) + reg

#Regularized gradient function
def regGradient(theta, X, y, _lambda = 0.1):
    m, n = X.shape
    theta = theta.reshape((n, 1))
    y = y.reshape((m, 1))
    h = sigmoid(X.dot(theta))
    tmp = np.copy(theta)
    tmp[0] = 0
    reg = _lambda*tmp /m

    return ((1 / m) * X.T.dot(h - y)) + reg

#Optimal theta 
def logisticRegression(X, y, theta):
    result = op.minimize(fun = regCostFunction, x0 = theta, args = (X, y),
                         method = 'TNC', jac = regGradient)
    
    return result.x

In [ ]:
#Training

all_theta = np.zeros((k, n + 1))

#One vs all
i = 0
for flower in Species:
    #set the labels in 0 and 1
    tmp_y = np.array(y == flower, dtype = int)
    optTheta = logisticRegression(X, tmp_y, np.zeros((n + 1,1)))
    all_theta[i] = optTheta
    i += 1

In [ ]:
#Predictions
P = sigmoid(X.dot(all_theta.T)) #probability for each flower
p = [Species[np.argmax(P[i, :])] for i in range(m)]

#true positives
tp = sum(np.array(p == y, dtype = int))

print('Train Accuracy ', (tp / m) * 100, '%')

In [ ]:
#Confusion Matrix
cfm = confusion_matrix(y, p, labels = Species)

sb.heatmap(cfm, annot = True, xticklabels = Species, yticklabels = Species)